# Project

In [ ]:
import os
import numpy as np
import cv2  
from matplotlib import pyplot as plt
from collections import defaultdict
import pandas as pd

In [ ]:
# Read in images from a filepath as graycsale.
rootpath='./'


# Reads the images in the training set
df_img=pd.read_csv('train.csv')
df_img['img']=[None]*len(df_img['id'])
df_img.set_index('id',inplace=True)


#Creates dict with key = id, and a list with positions 0: x coordinate, 1: y coordinate, 2: array representing img
#dict_img=defaultdict(list)


'''
for i in range(len(df_coord['id'])):
    id_img=df_coord.loc[i,'id']
    x_coord=df_coord.loc[i,'x']
    y_coord=df_coord.loc[i,'y']
    dict_img[id_img].append(x_coord)
    dict_img[id_img].append(y_coord)
'''
    
img_name_list=[]
edge_img_list=
for i, filename in enumerate(os.listdir(os.path.join(rootpath, 'train/'))):
    #if i>1:
    #    break
    img = cv2.imread(os.path.join(rootpath, f'train/{filename}'),cv2.IMREAD_GRAYSCALE)
    #print(img)
    file_name_no_ext=filename.replace('.jpg','')
    img_name_list.append(file_name_no_ext)
    
    #dict_img[file_name_no_ext].append(img)
    df_img.loc[file_name_no_ext,'img']=[img]
    
    #lt.imshow(img_gray, cmap='gray')
    #lt.show()


print(df_img)

'''
# Display original image and scene image
plt.subplots(figsize=(15, 15)) 

plt.subplot(1,2,1)
plt.imshow(gray, cmap='gray')  
plt.title('Original Image')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(scene_gray, cmap='gray')  
plt.title('Scene Image')
plt.axis('off')

plt.show() 
'''


In [ ]:
sift = cv2.SIFT_create()

kp_list=[]
desc_list=[]
for m, i in enumerate(df_img.index):
    #if  m>1:
    #    break
    image=df_img.loc[i,'img']
    kp, des = sift.detectAndCompute(image,None) 
    #rint(kp)
    kp_list.append(kp)
    desc_list.append(des)

df_img['kps']=kp_list
df_img['desc']=desc_list

In [ ]:
df_img

In [ ]:
df_img_test=pd.read_csv('imagenames.csv')
df_img_test['img']=[None]*len(df_img_test['id'])
df_img_test.set_index('id',inplace=True)

test_img_name_list=[]
for i, filename in enumerate(os.listdir(os.path.join(rootpath, 'test/'))):
    #if i>1:
    #    break
    img = cv2.imread(os.path.join(rootpath, f'test/{filename}'),cv2.IMREAD_GRAYSCALE)
    #print(img)
    test_file_name_no_ext=filename.replace('.jpg','')
    test_img_name_list.append(test_file_name_no_ext)
    
    #dict_img[file_name_no_ext].append(img)
    df_img_test.loc[test_file_name_no_ext,'img']=img
    
#GENERATE KEYPOINTS AND DESCRIPTORS
    
kp_test_list=[]
desc_test_list=[]
for m, i in enumerate(df_img_test.index):
    #if  m>1:
    #    break
    image=df_img_test.loc[i,'img']
    kp, des = sift.detectAndCompute(image,None) 
    #rint(kp)
    kp_test_list.append(kp)
    desc_test_list.append(des)

df_img_test['kps']=kp_test_list
df_img_test['desc']=desc_test_list

df_img_test





In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

indices=df_img.index.to_list()

indices
X_train, X_dev, _, _=train_test_split(indices, indices, test_size= 0.01, random_state=1)



# Matching

In [ ]:


def find_matches(matcher,des1, des2):
    # Matching descriptor using KNN algorithm
    matches = matcher.knnMatch(des1,des2,k=2)

    # Store all good matches as per Lowe's Ratio test.
    ratio = 0.6
    good = []
    for m,n in matches:
        if m.distance < ratio*n.distance:
            good.append(m)
    return len(good)

def match(df, known, unknown):
    
    # FLANN parameters and initialize
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    
    df_img1=df.copy() #Copies original df
    
    array=np.full((len(X_train),len(X_dev)),np.nan)    
    df_matches=pd.DataFrame(array,columns=unknown)
    df_matches['ind']=known
    df_matches.set_index('ind', inplace=True)
    
    for u in unknown:
        desc2=df_img1.loc[u,'desc']
        
        for k in known:
            print(k)
            desc1=df_img1.loc[k,'desc']
            
            #print(desc1)
            print(desc2)
            df_matches.loc[k,u]= find_matches(flann, desc1, desc2)
    
    return df_matches

good=match(df_img, X_train, X_dev)
            


good      
            
    



#df_img1['matches']=df_img1['matches'].map(lambda x: match(x))




In [ ]:
good

In [ ]:

from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor

lista=[]
lista.append([X_train])

train_descriptors=[]
y1_train=[]
y2_train=[]
for k in X_train:
    train_descriptors.append(df_img.loc[k,'desc'])    
    y1_train.append(df_img.loc[k,'x'])
    y2_train.append(df_img.loc[k,'y'])
    

index_none=[]
even_descr=[]
for i, t in enumerate(train_descriptors):
    if t is not None:
        even_descr.append(np.mean(t,axis=0))
    else:
        index_none.append(i)

#print(even_descr)
y1_t=[]
y2_t=[]
for i,k in enumerate(y1_train):
    if i not in index_none:
        y1_t.append(y1_train[i])
        y2_t.append(y2_train[i])
        

dev_descriptors=[]
y1_dev=[]
y2_dev=[]
index_none_dev=[]
for i, d in enumerate(X_dev):
    if d is not None:
        desc= df_img.loc[d,'desc']
        dev_descriptors.append(np.mean(desc,axis=0))
        y1_dev.append(df_img.loc[d,'x'])
        y2_dev.append(df_img.loc[d,'y'])
    else:
        index_none_dev.append(i)
    
dev_coord=np.column_stack((y1_dev,y2_dev))
    
train_coord=np.column_stack((y1_t,y2_t))

print('original lenght descr:',len(train_descriptors))
print('even lenght descr:',len(even_descr))


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

regressor=MultiOutputRegressor(GradientBoostingRegressor(random_state=0, criterion='mae',n_estimators=500)).fit(even_descr,train_coord)

result= regressor.predict(test_descriptors)

In [ ]:

def mean_abs_err(truth, preds):
    
    error=0
    for i, pred in enumerate(preds):
        error+= abs(truth[i][0]-preds[i][0])+abs(truth[i][1]-preds[i][1])
        
    return error/len(preds)

mae=mean_abs_err(dev_coord,result )
mae



In [ ]:
abs(-1)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor

X, y = make_regression(n_samples=10, n_targets=3, random_state=1)
#MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X, y).predict(X)
y.shape